In [ ]:
import pandas as pd
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
import keras_cv_attention_models as cv
tf.test.is_gpu_available()
import matplotlib.pyplot as plt

def cfp_model():
    model = tf.keras.Sequential()
    # model.add(cv.fastervit.FasterViT0(num_classes=0, input_shape=(299, 299, 3)))
    model.add(cv.lcnet.LCNet250(num_classes=0, input_shape=(299, 299, 3)))
    # model.add(tf.keras.applications.VGG16(include_top=False, input_shape=(299, 299, 3)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation("linear"))
    return model

from tensorflow.keras import backend as K

def pearson_corr(y_true, y_pred):
    mx = K.mean(y_true)
    my = K.mean(y_pred)
    xm = y_true - mx
    ym = y_pred - my
    r_num = K.sum(xm * ym)
    r_den = K.sqrt(K.sum(K.square(xm)) * K.sum(K.square(ym)))
    r = r_num / (r_den + K.epsilon())
    return r

model = cfp_model()
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_error',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5
    ),
    tf.keras.callbacks.CSVLogger(
        'training_log.csv',
        append=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',            
        monitor='val_mean_absolute_error',   
        save_best_only=True,                 
        save_weights_only=False,             
        verbose=1
    )
]

model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1),
    loss='mse',
    metrics=[
        tf.keras.metrics.MeanAbsoluteError(),
        pearson_corr
    ]
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_LC_ex.png', show_shapes=True, show_layer_names=True, dpi=400, expand_nested=True)

In [ ]:
history = model.fit(
    train_data,
    train_label,
    epochs=200,
    batch_size=64,
    validation_data=(test_data, test_label),
    callbacks=callbacks
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

def analyze_training_log(csv_path, save_prefix="training_plot", model_name="Model"):
    df = pd.read_csv(csv_path)
    lr_changes = df[df["lr"].diff() != 0][["epoch", "lr"]]

    fig, ax1 = plt.subplots(figsize=(10, 6))
    fig.suptitle(f"Training Analysis: {model_name}", fontsize=14, y=1.02)

    ax1.plot(df["epoch"], df["loss"], label="Train Loss", color='tab:blue', marker='o', zorder=2)
    ax1.plot(df["epoch"], df["val_loss"], label="Val Loss", color='tab:cyan', marker='o', zorder=2)
    ax1.set_ylabel("Loss (MSE)", color='tab:blue')
    ax1.set_xlabel("Epoch")
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax1b = ax1.twinx()
    ax1b.plot(df["epoch"], np.log1p(df["loss"]), label="Log(1+Train Loss)", color='tab:purple', linestyle='--', marker='^', zorder=1)
    ax1b.plot(df["epoch"], np.log1p(df["val_loss"]), label="Log(1+Val Loss)", color='tab:pink', linestyle='--', marker='^', zorder=1)
    ax1b.set_ylabel("Log(1 + Loss)", color='tab:purple')
    ax1b.tick_params(axis='y', labelcolor='tab:purple')

    for _, row in lr_changes.iterrows():
        epoch = row["epoch"]
        lr_val = row["lr"]
        ax1.axvline(x=epoch, color='gray', linestyle='--', alpha=0.6, zorder=0)
        ax1.text(epoch + 0.3, ax1.get_ylim()[1] * 0.95, f"lr={lr_val:.3f}", rotation=90, fontsize=8, color='gray', va='top')

    best_epoch = df["val_mean_absolute_error"].idxmin()
    best_val_loss = df.loc[best_epoch, "val_loss"]
    best_epoch_num = df.loc[best_epoch, "epoch"]
    ax1.plot(best_epoch_num, best_val_loss, marker='*', markersize=14, color='black', label='Val Loss @ Best MAE', zorder=3)
    ax1.text(best_epoch_num + 0.5, best_val_loss,
             f"Val Loss: {best_val_loss:.2f}\n(Epoch {best_epoch_num})",
             fontsize=10, color='black', zorder=3)

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax1b.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=2)

    ax1.set_title("Loss and Log Loss over Epochs with Learning Rate Annotations")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_loss.png", dpi=500, bbox_inches='tight')
    plt.show()

    fig, ax2 = plt.subplots(figsize=(10, 6))
    fig.suptitle(f"Training Analysis: {model_name}", fontsize=14, y=1.02)

    ax2.plot(df["epoch"], df["mean_absolute_error"], label="Train MAE", color='tab:red', marker='o')
    ax2.plot(df["epoch"], df["val_mean_absolute_error"], label="Val MAE", color='tab:orange', marker='o')
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("MAE", color='tab:red')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    ax2b = ax2.twinx()
    ax2b.plot(df["epoch"], df["pearson_corr"], label="Train Pearson r", color='tab:green', marker='s')
    ax2b.plot(df["epoch"], df["val_pearson_corr"], label="Val Pearson r", color='darkcyan', marker='s')
    ax2b.set_ylabel("Pearson r", color='tab:green')
    ax2b.tick_params(axis='y', labelcolor='tab:green')

    for _, row in lr_changes.iterrows():
        epoch = row["epoch"]
        lr_val = row["lr"]
        ax2.axvline(x=epoch, color='gray', linestyle='--', alpha=0.6)
        ax2.text(epoch + 0.3, ax2.get_ylim()[1] * 0.95, f"lr={lr_val:.3f}", rotation=90, fontsize=8, color='gray', va='top')

    best_val_mae = df.loc[best_epoch, "val_mean_absolute_error"]
    ax2.plot(best_epoch_num, best_val_mae, marker='*', markersize=12, color='black', label='Best Val MAE')
    ax2.text(best_epoch_num + 0.5, best_val_mae,
             f"Best MAE: {best_val_mae:.2f}\n(Epoch {best_epoch_num})",
             fontsize=10, color='black')

    lines3, labels3 = ax2.get_legend_handles_labels()
    lines4, labels4 = ax2b.get_legend_handles_labels()
    ax2.legend(lines3 + lines4, labels3 + labels4, loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=2)

    ax2.set_title("Model Performance over Epochs with Learning Rate Annotations")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_performance.png", dpi=500, bbox_inches='tight')
    plt.show()

analyze_training_log("./training_log.csv", model_name="VGG16")

In [ ]:
save_prefix="LCNet_NTUH"

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np

preds = model.predict(test_data).ravel()
truth = test_label.ravel()

mae = mean_absolute_error(truth, preds)
mse = mean_squared_error(truth, preds)
r2 = r2_score(truth, preds)
r, _ = pearsonr(truth, preds)

reg = LinearRegression().fit(truth.reshape(-1, 1), preds)
reg_line = reg.predict(np.array([truth.min(), truth.max()]).reshape(-1, 1))

plt.figure(figsize=(8, 8))
plt.scatter(truth, preds, alpha=0.5, edgecolors='k', label='Predictions')

plt.plot([truth.min(), truth.max()], [truth.min(), truth.max()],
         'r--', linewidth=2.5, label='Ideal (y = x)')

plt.plot([truth.min(), truth.max()], reg_line,
         'g--', linewidth=2.5, label='Fit Line')

plt.fill_between(
    [truth.min(), truth.max()],
    [truth.min() - mae, truth.max() - mae],
    [truth.min() + mae, truth.max() + mae],
    color='gray', alpha=0.4, label='±MAE'
)

plt.text(
    x=truth.min(), y=truth.max(),
    s=f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.3f}\nr = {r:.3f}",
    fontsize=13, ha='left', va='top', bbox=dict(facecolor='white', alpha=0.9, edgecolor='black')
)

plt.xlabel("True Values", fontsize=12)
plt.ylabel("Predicted Values", fontsize=12)
plt.title("True vs. Predicted", fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True)
plt.axis('equal')
plt.tight_layout()
plt.savefig(f"{save_prefix}_performance_scatter.png", dpi=500, bbox_inches='tight')
plt.show()


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import random

backbone = model.layers[0]
conv_output = backbone.get_layer("stack6_block2_output_hard_swish").output

x = tf.keras.layers.Flatten(name="flatten")(conv_output)
for layer_name in ["dense", "dropout", "re_lu", "batch_normalization",
                   "dense_1", "dropout_1", "re_lu_1", "batch_normalization_1",
                   "dense_2", "activation"]:
    x = model.get_layer(layer_name)(x)
output = x
grad_model = tf.keras.models.Model(inputs=backbone.input, outputs=[conv_output, output])

def compute_gradcam(image, label_value):
    img_input = np.expand_dims(image, axis=0).astype(np.float32)
    with tf.GradientTape() as tape:
        conv_out, prediction = grad_model(img_input)
        loss = tf.abs(prediction[0][0] - label_value)
    grads = tape.gradient(loss, conv_out)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.reduce_sum(pooled_grads * conv_out[0], axis=-1).numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(image[:, :, ::-1], 0.6, heatmap_color, 0.4, 0)[:, :, ::-1]
    return heatmap, overlay

save_dir = "gradcam_compare_bio_vs_chr_random"
os.makedirs(save_dir, exist_ok=True)

random_indices = random.sample(range(len(ex_data)), 200)

for idx in tqdm(random_indices):
    img = ex_data[idx]
    heatmap_bio, overlay_bio = compute_gradcam(img, ex_bio_label[idx])
    heatmap_chr, overlay_chr = compute_gradcam(img, ex_label[idx])

    fig, axs = plt.subplots(2, 3, figsize=(12, 6))
    axs[0, 0].imshow(img)
    axs[0, 0].set_title(f"Original\nBio: {ex_bio_label[idx]:.1f}, Chr: {ex_label[idx]:.1f}")
    axs[0, 0].axis("off")

    axs[0, 1].imshow(heatmap_chr, cmap='jet')
    axs[0, 1].set_title("Chronological Heatmap")
    axs[0, 1].axis("off")

    axs[0, 2].imshow(overlay_chr)
    axs[0, 2].set_title("Chronological Overlay")
    axs[0, 2].axis("off")

    axs[1, 1].imshow(heatmap_bio, cmap='jet')
    axs[1, 1].set_title("Biological Heatmap")
    axs[1, 1].axis("off")

    axs[1, 2].imshow(overlay_bio)
    axs[1, 2].set_title("Biological Overlay")
    axs[1, 2].axis("off")

    axs[1, 0].axis("off")

    plt.tight_layout()
    fig.savefig(os.path.join(save_dir, f"compare_{idx}.png"), dpi=300)
    plt.close()

print(f"✅ 儲存完成，共 30 張，路徑：{save_dir}")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import random
import os

disease_df = pd.read_csv("../../Dataset/odir/full_df.csv").iloc[:, 7:15]
disease_df.columns = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

disease_name_map = {
    'N': 'Normal',
    'D': 'Diabetic Retinopathy',
    'G': 'Glaucoma',
    'C': 'Cataract',
    'A': 'AMD',
    'H': 'Hypertension',
    'M': 'Myopia',
    'O': 'Other Abnormality'
}

save_dir = "./gradcam_disease_examples"
os.makedirs(save_dir, exist_ok=True)

backbone = model.layers[0]
conv_output = backbone.get_layer("stack6_block2_output_hard_swish").output

x = tf.keras.layers.Flatten(name="flatten")(conv_output)
for layer_name in ["dense", "dropout", "re_lu", "batch_normalization",
                   "dense_1", "dropout_1", "re_lu_1", "batch_normalization_1",
                   "dense_2", "activation"]:
    x = model.get_layer(layer_name)(x)
output = x
grad_model = tf.keras.models.Model(inputs=backbone.input, outputs=[conv_output, output])

def plot_gradcam(img, model, save_path):
    img_input = np.expand_dims(img, axis=0).astype(np.float32)

    with tf.GradientTape() as tape:
        conv_out, prediction = model(img_input)
        loss = prediction[0][0]

    grads = tape.gradient(loss, conv_out)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.reduce_sum(pooled_grads * conv_out[0], axis=-1).numpy()

    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img[:, :, ::-1], 0.6, heatmap_color, 0.4, 0)[:, :, ::-1]

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(img)
    axs[0].set_title("Original")
    axs[0].axis("off")
    axs[1].imshow(heatmap, cmap='jet')
    axs[1].set_title("Grad-CAM Heatmap")
    axs[1].axis("off")
    axs[2].imshow(overlay)
    axs[2].set_title("Overlay")
    axs[2].axis("off")
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

for col in disease_df.columns:
    disease_name = disease_name_map[col]
    indices = np.where(disease_df[col].values == 1)[0]
    if len(indices) < 20:
        continue
    selected_indices = random.sample(list(indices), 20)
    disease_path = os.path.join(save_dir, disease_name.replace(" ", "_"))
    os.makedirs(disease_path, exist_ok=True)

    for i, idx in enumerate(selected_indices):
        img = odir_data[idx]
        save_path = os.path.join(disease_path, f"{disease_name}_{i}.png")
        plot_gradcam(img, grad_model, save_path)

generated_dirs = sorted(os.listdir(save_dir))
generated_dirs